In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xarray as xr

import joblib
import pickle

import pandas as pd
import numpy as np
import datetime as dt
import os
import shutil

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/REPPU/200/pbig5min.dat' #Ubuntu
path = r'/home/sachin/Documents/NIPR/Research/Data/REPPU/pbig5min.dat' #Server

#read the REPPU data
with open (path) as f:
    rectype = np.dtype(np.float32)
    reppu_data = np.fromfile(f, rectype) #size = 109,900,800

reppu_data = reppu_data.reshape(-1,30,80)
reppu_data.shape

(45792, 30, 80)

In [3]:
mhd_data = pd.read_csv('mhd_dates.csv')
expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
    for start, end in zip(mhd_data['start'], mhd_data['end'])])

In [4]:
time_res = 5

if time_res == 1:
    time_step = 1440
else :
    time_step = 288

In [8]:
data_reshaped = reppu_data.reshape(len(expanded_dt), time_step, 30, 80) 

# Define coordinates
time = np.arange(time_step)
lat = np.linspace(50, 90, 30) #30 intervals between 53.1° to 89.7°
lon = np.linspace(1, 360, 80) #80 intervals between 1.6° to 357.6°

# Create 'dt' variable combining dates and five-minute intervals
dt = []
for day in expanded_dt:
    for t in time:
        dt.append(day + pd.Timedelta(minutes=t*time_res))
dt = np.array(dt) #convert from list to numpy array

# Create xarray Dataset
ds = xr.Dataset({'potential': (['dt', 'lat', 'lon'], data_reshaped.reshape(-1, 30, 80))},
coords={'dt': dt, 'lat': lat, 'lon': lon})

# Add potential and units
ds['potential'].attrs['units'] = 'kV'
ds['potential'] = ds['potential'] * 1e-3 # Convert to kV
ds

<xarray.Dataset>
Dimensions:    (dt: 45792, lat: 30, lon: 80)
Coordinates:
  * dt         (dt) datetime64[ns] 2021-05-10 ... 2022-07-31T23:55:00
  * lat        (lat) float64 50.0 51.38 52.76 54.14 ... 85.86 87.24 88.62 90.0
  * lon        (lon) float64 1.0 5.544 10.09 14.63 ... 346.4 350.9 355.5 360.0
Data variables:
    potential  (dt, lat, lon) float32 0.6909 0.507 0.3146 ... -22.67 -22.71

In [9]:
omni_df = pd.read_csv('omni_mhd_5min.csv')
#omni_df = pd.read_csv(omni_mhd_path+'omni_mhd_5min.csv')
omni_df.set_index('dt', inplace=True) #set the datetime as the index
omni_df = omni_df.ffill().bfill()
omni_df = omni_df.dropna() #drop any remaining NaNs
omni_df

omni_ds = xr.Dataset(omni_df)
omni_ds['dt'] = pd.to_datetime(omni_ds['dt']) #convert the index to datetime

#merge OMNI with REPPU data
reppu_omni_ds = ds.merge(omni_ds, join='outer')

#select date range
reppu_omni_ds = reppu_omni_ds.sortby('dt')
reppu_omni_ds

<xarray.Dataset>
Dimensions:         (dt: 45792, lat: 30, lon: 80)
Coordinates:
  * dt              (dt) datetime64[ns] 2021-05-10 ... 2022-08-19T23:55:00
  * lat             (lat) float64 50.0 51.38 52.76 54.14 ... 87.24 88.62 90.0
  * lon             (lon) float64 1.0 5.544 10.09 14.63 ... 350.9 355.5 360.0
Data variables:
    potential       (dt, lat, lon) float32 0.6909 0.507 0.3146 ... -15.45 -15.47
    BY_GSE          (dt) float64 4.72 4.35 4.61 4.65 ... -2.49 -2.49 -2.04 -1.92
    BZ_GSE          (dt) float64 6.24 6.55 6.56 6.89 ... -4.47 -4.47 -4.64 -4.5
    flow_speed      (dt) float64 344.0 344.0 345.0 344.0 ... 655.0 653.0 658.0
    proton_density  (dt) float64 12.11 11.44 12.65 12.63 ... 6.22 6.22 5.59 5.34
    tilt_angle      (dt) float64 0.253 0.2497 0.2465 ... 0.1837 0.1769 0.1735

In [ ]:
reppu_omni_ds['MLT'] 

In [10]:
df = reppu_omni_ds.to_dataframe()
df

potential  BY_GSE  BZ_GSE  flow_speed  \
dt                  lat  lon                                                 
2021-05-10 00:00:00 50.0 1.000000     0.690854    4.72    6.24       344.0   
                         5.544304     0.507001    4.72    6.24       344.0   
                         10.088608    0.314646    4.72    6.24       344.0   
                         14.632911    0.115738    4.72    6.24       344.0   
                         19.177215   -0.087280    4.72    6.24       344.0   
...                                        ...     ...     ...         ...   
2022-08-19 23:55:00 90.0 341.822785 -15.385611   -1.92   -4.50       658.0   
                         346.367089 -15.406006   -1.92   -4.50       658.0   
                         350.911392 -15.427092   -1.92   -4.50       658.0   
                         355.455696 -15.448745   -1.92   -4.50       658.0   
                         360.000000 -15.470826   -1.92   -4.50       658.0   

                                     proton_density  tilt_angle  
dt                  lat  lon                                     
2021-05-10 00:00:00 50.0 1.000000             12.11    0.253032  
                         5.544304             12.11    0.253032  
                         10.088608            12.11    0.253032  
                         14.632911            12.11    0.253032  
                         19.177215            12.11    0.253032  
...                                             ...         ...  
2022-08-19 23:55:00 90.0 341.822785            5.34    0.173477  
                         346.367089            5.34    0.173477  
                         350.911392            5.34    0.173477  
                         355.455696            5.34    0.173477  
                         360.000000            5.34    0.173477  

[109900800 rows x 6 columns]